In [10]:
import os
import numpy as np
from sklearn.metrics import f1_score
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Replace these paths with your actual dataset paths
test_data_path = 'D:/project/dataset/noise_reduced_data'
model_path = 'D:/project/driver_drowsiness/models/model.h5'

# Specify image data generator parameters
batch_size = 32
img_size = (80, 80)

# Create and configure an ImageDataGenerator for the test data
test_datagen = ImageDataGenerator(rescale=1./255)

# Create a generator for the test data
test_data_gen = test_datagen.flow_from_directory(
    test_data_path,
    target_size=img_size, batch_size=batch_size, class_mode='categorical', shuffle=False
)

# Load the pre-trained model
model = load_model(model_path)

# Get true labels for the test data
true_labels = test_data_gen.classes

# Get predictions from the model
predictions = model.predict(test_data_gen)

# Convert predictions to binary labels
predicted_labels = np.argmax(predictions, axis=1)

# Calculate the F1 score
f1 = f1_score(true_labels, predicted_labels)

print(f"F1 Score: {f1}")


Found 84898 images belonging to 2 classes.
2654/2654 [==============================] - 776s 288ms/step
F1 Score: 0.9426784059314178


In [10]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
model = load_model(r'D:/project/driver_drowsiness/models/model.h5') 
frame = cv2.imread(r"C:\Users\Sai Rohit\Pictures\Camera Roll\test4.jpg")
frame = cv2.convertScaleAbs(frame, alpha=1.2, beta=20)
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))
c=0

for (ex, ey, ew, eh) in eyes:
    eye = frame[ey:ey + eh, ex:ex + ew]
    eye = cv2.resize(eye, (80, 80))
    eye = eye / 255.0
    eye = np.expand_dims(eye, axis=0)

    # Make predictions using the eye model
    prediction = model.predict(eye)

    # Check the prediction
    if prediction[0][1] > 0.6:
        status = 'Open'
        c=1

    print(status)
    break
if(c==0):
    print("closed")




1/1 [==============================] - 3s 3s/step
Open


In [11]:
import cv2
import numpy as np
from pygame import mixer
from tensorflow.keras.models import load_model

mixer.init()
sound = mixer.Sound(r'D:\project\driver_drowsiness\alarm.wav')
cap = cv2.VideoCapture(0)
Score = 0

# Load pre-trained face detector
face_detector = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load pre-trained eye detector
eye_detector = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Load your pre-trained model
model = load_model(r'D:\project\driver_drowsiness\models\model.h5')

frame_skip = 2
frame_count = 0

while True:
    ret, frame = cap.read()
    frame_count += 1

    if frame_count % frame_skip != 0:
        continue

    height, width = frame.shape[0:2]
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Increase brightness
    frame = cv2.convertScaleAbs(frame, alpha=1.2, beta=20)

    # Use pre-trained face detector
    faces = face_detector.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    cv2.rectangle(frame, (0, height-50), (200, height), (0, 0, 0), thickness=cv2.FILLED)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)  # Draw rectangle around the face

        roi_gray = gray[y:y+h, x:x+w]
        eyes = eye_detector.detectMultiScale(roi_gray)

        for (ex, ey, ew, eh) in eyes:
            eye = frame[y+ey:y+ey+eh, x+ex:x+ex+ew]
            eye = cv2.resize(eye, (80, 80))
            eye = eye / 255.0
            eye = np.expand_dims(eye, axis=0)

            # Model prediction
            prediction = model.predict(eye)

            if prediction[0][1] > 0.6:  # Adjust the threshold based on your model's characteristics
                cv2.putText(frame, 'open', (10, height-20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,
                            fontScale=1, color=(255, 255, 255), thickness=1, lineType=cv2.LINE_AA)
                Score = max(0, Score - 1)
            else:
                cv2.putText(frame, 'closed', (10, height-20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,
                            fontScale=1, color=(255, 255, 255), thickness=1, lineType=cv2.LINE_AA)
                Score += 1
                if Score > 5 :  # Only play the sound if at least one eye is detected
                    try:
                        sound.play()
                    except:
                        pass

            cv2.putText(frame, 'Score' + str(Score), (100, height-20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,
                        fontScale=1, color=(255, 255, 255), thickness=1, lineType=cv2.LINE_AA)

    cv2.imshow('frame', frame)

    if cv2.waitKey(33) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
1/1 [==============================] - 0s 59ms/step


In [17]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load the pre-trained eye cascade classifier
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Load the pre-trained eye detection model
model = load_model(r'D:/project/driver_drowsiness/models/model.h5')

# Video path
video_path = r'C:\Users\Sai Rohit\Pictures\Camera Roll\video test 3.mp4'  

# Open the video file
cap = cv2.VideoCapture(video_path)

# Set the frame skip parameters
frame_skip = 10
frame_count = 0
res={'Open':0,'Closed':0}
while True:
    ret, frame = cap.read()
    frame_count += 1

    if frame_count % frame_skip != 0:
        continue

    if not ret:
        break  # Break the loop if the video ends

    # Increase brightness
    frame = cv2.convertScaleAbs(frame, alpha=1.2, beta=20)

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect eyes in the frame
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

    for (ex, ey, ew, eh) in eyes:
        eye = frame[ey:ey + eh, ex:ex + ew]
        eye = cv2.resize(eye, (80, 80))
        eye = eye / 255.0
        eye = np.expand_dims(eye, axis=0)

        # Make predictions using the eye model
        prediction = model.predict(eye)

        # Check the prediction and print the result
        if prediction[0][1] > 0.6:
            status = 'Open'
        else:
            status = 'Closed'
        res[status]+=1
        print(f"Eye status at {cap.get(cv2.CAP_PROP_POS_MSEC) / 1000:.1f} seconds: {status}")
print(res['Closed'],res['Open'])
if(res['Closed']<(res['Open']//2)):
    print("Overal Result the eyes are : Open")
else:
    print("Overal Result the eyes are : Closed")



1/1 [==============================] - 3s 3s/step
Eye status at 0.6 seconds: Open
1/1 [==============================] - 0s 112ms/step
Eye status at 0.6 seconds: Open
1/1 [==============================] - 0s 112ms/step
Eye status at 1.3 seconds: Open
1/1 [==============================] - 0s 82ms/step
Eye status at 1.3 seconds: Open
1/1 [==============================] - 0s 98ms/step
Eye status at 2.0 seconds: Open
1/1 [==============================] - 0s 98ms/step
Eye status at 2.0 seconds: Closed
1/1 [==============================] - 0s 95ms/step
Eye status at 2.0 seconds: Closed
1/1 [==============================] - 0s 95ms/step
Eye status at 2.0 seconds: Closed
1/1 [==============================] - 0s 91ms/step
Eye status at 2.0 seconds: Open
1/1 [==============================] - 0s 79ms/step
Eye status at 2.6 seconds: Open
1/1 [==============================] - 0s 115ms/step
Eye status at 2.6 seconds: Open
1/1 [==============================] - 0s 93ms/step
Eye status at 2.6